# Importações

In [94]:
from calendar import month

import pandas as pd
import datetime
import calendar
import locale
import numpy as np
import glob
import os
import math

from jinja2 import Undefined

locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')


'pt_BR.UTF-8'

# Carregar datasets básicos

In [95]:
consumo = pd.read_csv('./dados/consumo.csv', sep=";", header=0)
consumo["DATA INICIO"] = pd.to_datetime(consumo['DATA INICIO'], format='%d/%m/%Y')
consumo["DATA FIM"] = pd.to_datetime(consumo['DATA FIM'], format='%d/%m/%Y')
feriados = pd.read_csv('dados/calendarios.csv', sep=";", header=0)
cursos_sup = pd.read_csv('./dados/cursos_superiores.csv', sep=";", header=0)
cursos_tec = pd.read_csv('./dados/cursos_tecnicos.csv', sep=";", header=0)


# Agrupar Datasets


In [96]:

# Agrupando os dados por campus 
grupos_consumo = consumo.groupby('CAMPUS')
grupos_feriados = feriados.groupby('CAMPUS')
grupos_cursos_sup = cursos_sup.query('Modalidade == "Educação Presencial"').groupby('CAMPUS')
grupos_cursos_tec = cursos_tec.query('NO_MODALIDADE == "EDUCAÇÃO PRESENCIAL"').groupby('CAMPUS')

# Criando subdatasets para cada campus 
subdf_consumo = {campus: dados for campus, dados in grupos_consumo}
subdf_feriados = {campus: dados for campus, dados in grupos_feriados}
subdf_cursos_sup = {campus: dados for campus, dados in grupos_cursos_sup}
subdf_cursos_tec = {campus: dados for campus, dados in grupos_cursos_tec}



# Transformar Dados de Consumo Mensais em Diários

In [97]:
dataframes = []
dataframes_ignorados = []
for campus, df_consumo in subdf_consumo.items():
    dt_min = subdf_consumo[campus]["DATA INICIO"].min()
    dt_max = subdf_consumo[campus]["DATA FIM"].max()

    if dt_min.day != 1:
        next_month = (dt_min.replace(day=28) + pd.DateOffset(days=4)).replace(day=1)
        dt_min = next_month

    if dt_max != (dt_max + pd.offsets.MonthEnd(0)):
        dt_max = dt_max + pd.offsets.MonthEnd(-1)

    datas = pd.date_range(start=dt_min, end=dt_max)
    df = pd.DataFrame({'DATA': datas})
    df["CAMPUS"] = campus

    for index, row in df_consumo.iterrows():
        dt_inicio = row['DATA INICIO']
        dt_fim = row['DATA FIM']
        nr_dias = (dt_fim - dt_inicio).days + 1

        mascara = (df['DATA'] >= pd.to_datetime(dt_inicio)) & (df['DATA'] <= pd.to_datetime(dt_fim))
        df.loc[mascara, "CONSUMO"] = row["CONSUMO"] / nr_dias

    if "PARANAGUÁ" in campus:
        dataframes_ignorados.append(df)
    else:
        dataframes.append(df)

df_merged = pd.merge(dataframes_ignorados[0], dataframes_ignorados[1], on='DATA', how='inner',
                     suffixes=('_df1', '_df2'))
df_merged["CONSUMO"] = df_merged["CONSUMO_df1"].fillna(0) + df_merged["CONSUMO_df2"].fillna(0)
df_merged["CAMPUS"] = "PARANAGUÁ"
df_merged = df_merged.drop(["CONSUMO_df1", "CONSUMO_df2", "CAMPUS_df1", "CAMPUS_df2"], axis=1)
dataframes.append(df_merged)

df_consumo_normalizado = pd.concat(dataframes, ignore_index=True)

df_consumo_normalizado.to_csv("./dados/dados_consumo_normalizados.csv", sep=";", decimal=".", index=False)

grupos_consumo_normalizado = df_consumo_normalizado.groupby('CAMPUS')
subdf_consumo_normalizado = {campus: dados for campus, dados in grupos_consumo_normalizado}


# Ajustar Dados Climáticos

In [98]:
df_clima = pd.read_csv("./dados/clima.csv", sep=";", header=0, decimal=".")
df_clima_londrina = df_clima.query('CAMPUS == "LONDRINA"')

df_clima_londrina.loc[df_clima_londrina['CAMPUS'] == "LONDRINA", 'CAMPUS'] = "LONDRINA - NORTE"
df_clima.loc[df_clima['CAMPUS'] == "LONDRINA", 'CAMPUS'] = "LONDRINA - CENTRO"

df_clima = pd.concat([df_clima, df_clima_londrina], ignore_index=True)

df_clima["DATA"] = pd.to_datetime(df_clima["DATA"], format="%d/%m/%Y")
df_clima.to_csv("./dados/dados_climaticos.csv", sep=";", decimal=".", index=False)

# Criar Datasets conforme Calendários Acadêmicos

In [99]:

for campus, df_feriados in subdf_consumo_normalizado.items():
    dt_min = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).min()
    dt_max = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).max()

    dt_inicio = datetime.date(year=2010, month=1, day=1)
    dt_fim = datetime.date.today()

    datas = pd.date_range(start=dt_inicio, end=dt_fim)
    df = pd.DataFrame({'DATA': datas})
    df["CAMPUS"] = campus

    for index, row in df.iterrows():
        data = row['DATA'].date()
        dia = data.day
        mes = data.month
        ano = data.year

        rows_data_fixa = feriados.loc[feriados["DATA INICIO"] == f"{dia}/{mes}"]
        rows_data_flexivel = feriados.loc[feriados["DATA INICIO"] == f"{dia:02}/{mes:02}/{ano:04}"]

        if not rows_data_fixa.empty:
            for _, row_data_fixa in rows_data_fixa.iterrows():
                if campus in row_data_fixa["CAMPUS"] or row_data_fixa["CAMPUS"] == "*":
                    [dia_final_fixo, mes_final_fixo] = str(row_data_fixa["DATA FIM"]).split("/")
                    data_final_fixa = datetime.date(ano, int(mes_final_fixo), int(dia_final_fixo))
                    mascara = (df['DATA'] >= pd.to_datetime(data)) & (df['DATA'] <= pd.to_datetime(data_final_fixa))
                    df.loc[mascara, row_data_fixa["MOTIVO"]] = 1

        if not rows_data_flexivel.empty:
            for _, row_data_flexivel in rows_data_flexivel.iterrows():
                if campus in row_data_flexivel["CAMPUS"] or row_data_flexivel["CAMPUS"] == "*":
                    [dia_final_flexivel, mes_final_flexivel, ano_final_flexivel] = str(
                        row_data_flexivel["DATA FIM"]).split("/")
                    data_final_flexivel = datetime.date(int(ano_final_flexivel), int(mes_final_flexivel),
                                                        int(dia_final_flexivel))
                    mascara = (df['DATA'] >= pd.to_datetime(data)) & (df['DATA'] <= pd.to_datetime(data_final_flexivel))
                    df.loc[mascara, row_data_flexivel["MOTIVO"]] = 1

    df = df.fillna(0)
    df = df.loc[(df['DATA'] >= dt_min) & (df['DATA'] <= dt_max)]
    df.to_csv(f"./dados/calendarios/{campus}.csv", index=False, sep=";", decimal=".")


# Criar Dataset de Dados Temporais

In [100]:
pasta = './dados/calendarios/'

# Obter uma lista de todos os arquivos .csv na pasta
arquivos_csv = glob.glob(os.path.join(pasta, "*.csv"))
# Criar uma lista para armazenar os DataFrames
dataframes = []

# Iterar sobre a lista de arquivos .csv e ler cada um em um DataFrame
for arquivo in arquivos_csv:
    df = pd.read_csv(arquivo, sep=";", header=0)
    df['CAMPUS'] = os.path.basename(arquivo.replace(".csv", ""))
    df['DATA'] = pd.to_datetime(df['DATA'])
    df['DIA_DA_SEMANA'] = df['DATA'].dt.strftime('%a')
    df['MÊS'] = df['DATA'].dt.strftime('%b')
    df['ANO'] = df['DATA'].dt.strftime('%Y')
    df = pd.get_dummies(df, columns=['DIA_DA_SEMANA', "MÊS", "ANO"])
    df = df.fillna(False)
    df[df.drop(["CAMPUS", "DATA"], axis=1).columns] = df[df.drop(["CAMPUS", "DATA"], axis=1).columns].astype(int)
    dataframes.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_completo = pd.concat(dataframes, ignore_index=True)
df_completo = df_completo.fillna(0)

df_completo.to_csv("./dados/dados_temporais.csv", sep=";", decimal=".", index=False)


# Criar Datasets conforme Cursos Superiores (E-MEC)

In [101]:
dataframes = []
for campus, df_cursos_sup in subdf_cursos_sup.items():
    dt_min = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).min()
    dt_max = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).max()

    datas = pd.date_range(start=pd.to_datetime(df_cursos_sup["Início Funcionamento"], format='%d/%m/%Y').min(),
                          end=datetime.date.today())
    df = pd.DataFrame({'DATA': datas})
    df["CAMPUS"] = campus
    df["CURSOS_GRAD_MATUTINO"] = 0
    df["CURSOS_GRAD_VESPERTINO"] = 0
    df["CURSOS_GRAD_NOTURNO"] = 0
    df["CURSOS_POS"] = 0

    for index, row in df_cursos_sup.iterrows():
        dt_inicio = pd.to_datetime(row["Início Funcionamento"], format='%d/%m/%Y')
        if row["Fim Funcionamento"] is not None and not pd.isnull(row["Fim Funcionamento"]):
            dt_fim = pd.to_datetime(row["Fim Funcionamento"], format='%d/%m/%Y')
        else:
            dt_fim = pd.to_datetime(datetime.date.today())

        if row["Grau"] in ["Bacharelado", "Tecnológico", "Licenciatura"]:
            if row["Quantitativo de Vagas - Integral"] > 0:
                df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) &
                       (df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_GRAD_MATUTINO"] += 1
                df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) &
                       (df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_GRAD_VESPERTINO"] += 1
            else:
                if row["Quantitativo de Vagas - Matutino"] > 0:
                    df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) &
                           (df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_GRAD_MATUTINO"] += 1

                if row["Quantitativo de Vagas - Vespertino"] > 0:
                    df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) &
                           (df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_GRAD_VESPERTINO"] += 1

            if row["Quantitativo de Vagas - Noturno"] > 0:
                df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) &
                       (df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_GRAD_NOTURNO"] += 1

        elif row["Grau"] in ["Especialização", "Mestrado"]:
            df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) &
                   (df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_POS"] += 1

    df = df.fillna(0)
    df = df.loc[(df['DATA'] >= dt_min) & (df['DATA'] <= dt_max)]
    df.to_csv(f"./dados/cursos_superiores/{campus}.csv", index=False, sep=";", decimal=".")
    dataframes.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_completo = pd.concat(dataframes, ignore_index=True)
df_completo = df_completo.fillna(0)

df_completo.to_csv("./dados/dados_cursos_superiores.csv", sep=";", decimal=".",index=False)


# Criar Datasets conforme Cursos Técnicos (SISTEC)

In [102]:
dataframes = []
for campus, df_cursos_tec in subdf_cursos_tec.items():
    dt_min = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).min()
dt_max = pd.to_datetime(subdf_consumo_normalizado[campus]["DATA"]).max()

datas = pd.date_range(start=pd.to_datetime(df_cursos_tec["DATA INICIO"], format='%d/%m/%Y').min(),
                      end=datetime.date.today())
df = pd.DataFrame({'DATA': datas})
df["CAMPUS"] = campus
df["CURSOS_TEC_CONCOMITANTE"] = 0
df["CURSOS_TEC_INTEGRADO"] = 0
df["CURSOS_TEC_SUBSEQUENTE"] = 0

for index, row in df_cursos_tec.iterrows():
    dt_inicio = pd.to_datetime(row["DATA INICIO"], format='%d/%m/%Y')
dt_fim = pd.to_datetime(row["DATA FIM"], format='%d/%m/%Y')

if row["NO_TIPO_OFERTA"] == "CONCOMITANTE":
    df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) & (
            df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_TEC_CONCOMITANTE"] += 1
elif row["NO_TIPO_OFERTA"] == "INTEGRADO":
    df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) & (
            df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_TEC_INTEGRADO"] += 1
elif row["NO_TIPO_OFERTA"] == "SUBSEQUENTE":
    df.loc[(df['DATA'] >= pd.to_datetime(dt_inicio)) & (
            df['DATA'] <= pd.to_datetime(dt_fim)), "CURSOS_TEC_SUBSEQUENTE"] += 1

df = df.fillna(0)
df = df.loc[(df['DATA'] >= dt_min) & (df['DATA'] <= dt_max)]
df.to_csv(f"./dados/cursos_tecnicos/{campus}.csv", index=False, sep=";", decimal=".")
dataframes.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df_completo = pd.concat(dataframes, ignore_index=True)
df_completo = df_completo.fillna(0)

df_completo.to_csv("./dados/dados_cursos_tecnicos.csv", sep=";", decimal=".", index=False)


# Mesclar Datasets

In [104]:
dados_consumo = pd.read_csv("./dados/dados_consumo_normalizados.csv", header=0, sep=";", decimal=".")
dados_climaticos = pd.read_csv("./dados/dados_climaticos.csv", header=0, sep=";", decimal=".")
dados_temporais = pd.read_csv("./dados/dados_temporais.csv", header=0, sep=";", decimal=".")
dados_cursos_tecnicos = pd.read_csv("./dados/dados_cursos_tecnicos.csv", header=0, sep=";", decimal=".")
dados_cursos_superiores = pd.read_csv("./dados/dados_cursos_superiores.csv", header=0, sep=";", decimal=".")

df_merged = (dados_consumo
             .merge(dados_climaticos, on=['DATA', "CAMPUS"], how='left')
             .merge(dados_temporais, on=['DATA', "CAMPUS"], how='left')
             .merge(dados_cursos_tecnicos, on=['DATA', "CAMPUS"], how='left')
             .merge(dados_cursos_superiores, on=['DATA', "CAMPUS"], how='left'))

df_merged[dados_cursos_superiores.columns] = df_merged[dados_cursos_superiores.columns].fillna(0)
df_merged[dados_cursos_tecnicos.columns] = df_merged[dados_cursos_tecnicos.columns].fillna(0)

df_merged['MES'] = pd.to_datetime(df_merged['DATA']).dt.month

df_merged = df_merged.groupby(['CAMPUS', "MES"]).agg({
    'CONSUMO': 'sum',
    'TEMP_MIN_MED_MENS': ("TEMP_MIN", 'mean'),
    'TEMP_MIN_MIN_MENS': ("TEMP_MIN", 'min'),
    'TEMP_MIN_MAX_MENS': ("TEMP_MIN", 'max'),
    'TEMP_MIN_ACC_MENS': ("TEMP_MIN", 'sum'),
    'TEMP_MÉD_MED_MENS': ("TEMP_MÉD", 'mean'),
    'TEMP_MÉD_MIN_MENS': ("TEMP_MÉD", 'min'),
    'TEMP_MÉD_MAX_MENS': ("TEMP_MÉD", 'max'),
    'TEMP_MÉD_ACC_MENS': ("TEMP_MÉD", 'sum'),
    'TEMP_MAX_MED_MENS': ("TEMP_MAX", 'mean'),
    'TEMP_MAX_MIN_MENS': ("TEMP_MAX", 'min'),
    'TEMP_MAX_MAX_MENS': ("TEMP_MAX", 'max'),
    'TEMP_MAX_ACC_MENS': ("TEMP_MAX", 'sum'),
    'PRECIPITAÇÃO_MED_MENS': ("PRECIPITAÇÃO", 'mean'),
    'PRECIPITAÇÃO_MIN_MENS': ("PRECIPITAÇÃO", 'min'),
    'PRECIPITAÇÃO_MAX_MENS': ("PRECIPITAÇÃO", 'max'),
    'PRECIPITAÇÃO_ACC_MENS': ("PRECIPITAÇÃO", 'sum'),
    'FÉRIAS': 'sum',
    'FERIADO': 'sum',
    'COVID': 'sum',
    'GREVE': 'sum',
    'DIA_DA_SEMANA_dom': 'sum',
    'DIA_DA_SEMANA_seg': 'sum',
    'DIA_DA_SEMANA_ter': 'sum',
    'DIA_DA_SEMANA_qua': 'sum',
    'DIA_DA_SEMANA_qui': 'sum',
    'DIA_DA_SEMANA_sex': 'sum',
    'DIA_DA_SEMANA_sÃ¡b': 'sum',
    'MÊS_jan': 'mean',
    'MÊS_fev': 'mean',
    'MÊS_mar': 'mean',
    'MÊS_abr': 'mean',
    'MÊS_mai': 'mean',
    'MÊS_jun': 'mean',
    'MÊS_jul': 'mean',
    'MÊS_ago': 'mean',
    'MÊS_set': 'mean',
    'MÊS_out': 'mean',
    'MÊS_nov': 'mean',
    'MÊS_dez': 'mean',
    'AN0_2014': 'mean',
    'AN0_2015': 'mean',
    'AN0_2016': 'mean',
    'AN0_2017': 'mean',
    'AN0_2018': 'mean',
    'AN0_2019': 'mean',
    'AN0_2020': 'mean',
    'AN0_2021': 'mean',
    'AN0_2022': 'mean',
    'AN0_2023': 'mean',
    'AN0_2024': 'mean',
    'CURSOS_TEC_CONCOMITANTE': 'mean',
    'CURSOS_TEC_INTEGRADO': 'mean',
    'CURSOS_TEC_SUBSEQUENTE': 'mean',
    'CURSOS_GRAD_MATUTINO': 'mean',
    'CURSOS_GRAD_VESPERTINO': 'mean',
    'CURSOS_GRAD_NOTURNO': 'mean',
    'CURSOS_POS': 'mean'
})

df_merged.drop("MES", axis=1).to_csv("./dados/dados_mesclados.csv", sep=";", decimal=".", index=False)

df_merged

KeyError: "Column(s) ['AN0_2014', 'AN0_2015', 'AN0_2016', 'AN0_2017', 'AN0_2018', 'AN0_2019', 'AN0_2020', 'AN0_2021', 'AN0_2022', 'AN0_2023', 'AN0_2024', 'PRECIPITAÇÃO_ACC_MENS', 'PRECIPITAÇÃO_MAX_MENS', 'PRECIPITAÇÃO_MED_MENS', 'PRECIPITAÇÃO_MIN_MENS', 'TEMP_MAX_ACC_MENS', 'TEMP_MAX_MAX_MENS', 'TEMP_MAX_MED_MENS', 'TEMP_MAX_MIN_MENS', 'TEMP_MIN_ACC_MENS', 'TEMP_MIN_MAX_MENS', 'TEMP_MIN_MED_MENS', 'TEMP_MIN_MIN_MENS', 'TEMP_MÉD_ACC_MENS', 'TEMP_MÉD_MAX_MENS', 'TEMP_MÉD_MED_MENS', 'TEMP_MÉD_MIN_MENS'] do not exist"